In [ ]:
from lxml import html
import lxml
import time
import requests
import arrow
date_jour = arrow.now().format('YYYY-MM-DD')
date_jour

In [ ]:
url = 'https://labelleassiette.fr/villes-et-services'
response = requests.get(url)
content_source = response.text.encode("utf-8")
xml_doc = html.document_fromstring(content_source)
villes = xml_doc.xpath("//a[@class='cities-and-services-link']/@href")
villes[0:5]

In [ ]:
def pageContainsSuite(ville, page):
    url = 'https://labelleassiette.fr/chef-a-domicile/menus/' + ville
    params = {'gamme': 'allMenus',
             'searchType': 'private-chef',
             'country': 'FR',
             'experienceType': 'private-chef',
             'page': page}
    response = requests.get(url, params=params)
    content_source = response.text.encode("utf-8")
    xml_doc = html.document_fromstring(content_source)
    page_suite_existe = xml_doc.xpath("//@data-next-page-url")
    return page_suite_existe[0] != 'undefined'

pageContainsSuite('Brest', 3)

In [ ]:
def getChefsVillePage(ville, page):
    url = 'https://labelleassiette.fr/chef-a-domicile/menus/' + ville
    params = {'gamme': 'allMenus',
             'searchType': 'private-chef',
             'country': 'FR',
             'experienceType': 'private-chef',
             'page': page}
    response = requests.get(url, params=params)
    content_source = response.text.encode("utf-8")
    xml_doc = html.document_fromstring(content_source)
    chefs = xml_doc.xpath("//@data-chef-slug")
    return chefs

getChefsVillePage('Lille', 5)

In [ ]:
OUTPUT_PATH = '../_outputs/chefs_{}/'.format(date_jour)

In [ ]:
for ville in villes:
    chefs = []
    page = 1
    dernierePagePlusUn = False
    dernierePage = False
    ville = ville.split('/')[-1] # prend la ville sans le lien
    while not dernierePagePlusUn:
    # while testFin(page):
        print('Ville en cours = {}'.format(ville))
        print('Page en cours = {}'.format(page))
        more = getChefsVillePage(ville, page)
        chefs.extend(more) 
        page += 1
        if dernierePage :
            dernierePagePlusUn = True
        if not pageContainsSuite(ville, page):
            dernierePage = True
        time.sleep(2)
    with open("{}{}.txt".format(OUTPUT_PATH, ville), "w") as file:
        file.write("\n".join(chefs))